In [1]:
import uproot
import awkward as ak
import numpy as np
import math
from tqdm import tqdm
import hist
import matplotlib.pyplot as plt
import os
import subprocess
import vector
import gc
from itertools import product
import sys
sys.path.append('..')  
from Plot import Plot

vector.register_awkward() 
DATATYPE="data"
BASEDIR="/pbs/throng/training/nantes-m2-rps-exp/data"
SAMPLE_RUN = [291694, 290254, 290427, 290613, 290776, 291002, 291360, 291482, 291755, 292061]
MUON_MASS = 0.105
D_ABS = 500

if DATATYPE=="mc":
    gen_tree = f"{BASEDIR}/run{SAMPLE_RUN}.{DATATYPE}.root:genTree"
IS_MC=True if DATATYPE=="mc" else False
def data_file_path(run,is_mc=IS_MC,dest=BASEDIR):
    datatype="mc" if is_mc else "data"
    print({dest},"/run",{run},".",{datatype},".root")
    return f"{dest}/run{run}.{datatype}.root"

In [2]:
def getTracks(events):
    return ak.zip({ "px":events["Muon_Px"],
                    "py":events["Muon_Py"],
                    "pz":events["Muon_Pz"],
                    "E":events["Muon_E"],
                    "DCAx":events["Muon_xDCA"],
                    "DCAy":events["Muon_yDCA"],
                    "DCAz":events["Muon_zDCA"],
                    "charge":events["Muon_Charge"],
                    "thetaAbs":events["Muon_thetaAbs"],
                    "matched":events["Muon_matchedTrgThreshold"]},
                    with_name='Momentum4D')

In [3]:
def applySelection(data, criteria, plot=False, **kwargs):
    selected = data[criteria(data)]
    if plot:
        hist_before = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$|p|$'))
        hist_after = hist.Hist(hist.axis.Regular(bins=100,start=0,stop=20,name='$|p|$'))
        try:
            hist_before.fill(ak.flatten(data.pt))
            hist_after.fill(ak.flatten(selected.pt))
        except AttributeError:
            hist_before.fill(ak.flatten(getTracks(data).pt))
            hist_after.fill(ak.flatten(getTracks(selected).pt))
        #Plot(hist_before, hist_after, **kwargs)
    return selected

In [4]:
def get_pDCA_infos(goodTracks):
    DCA=np.sqrt(goodTracks.DCAx**2 + goodTracks.DCAy**2 + goodTracks.DCAz**2)
    p0=ak.where(goodTracks.thetaAbs < 3 , goodTracks.p - 3.0, goodTracks.p - 2.4)
    pDCA=p0*DCA
    
    N=3 #choice of resolution should be a number between 1 and 10
    sigmaAbs=ak.where(goodTracks.thetaAbs < 3, 80.0, 54.0) #factor used due to the crossed materials 
    pRes= 0.0004 * goodTracks.p
    sigmap= sigmaAbs/ (1 - ((N*pRes)/(1 + N*pRes)))

    thetaRes= 0.0005 * goodTracks.p
    sigmatheta= 535 * thetaRes

    #computing total resolution for pDCA
    sigmapDCA=np.sqrt(sigmap**2 + sigmatheta**2)

    goodTracks["pDCA"] = pDCA

    return pDCA, N, sigmapDCA

In [5]:
vhMagRaw = hist.Hist(hist.axis.Regular(bins=1000, start=0, stop=5, name='$|p|$'))

def analyzeEventsTree():
    events = uproot.concatenate([data_file_path(SAMPLE) for SAMPLE in SAMPLE_RUN])
    
    # Définition des fonctions de sélection
    def select_charge(events):
        charge = []
        for charges in events["Muon_Charge"]:
            if (-1 in charges) and (1 in charges):
                charge += [True]
            else:
                charge += [False]
        return charge
    
    def select_z(events):
        z = events["zVtx"]
        return (z < 10) & (z > -10)
    
    select_nb = lambda events: events["nMuons"] >= 2
    
    # Sélections pour les tracks
    select_pt = lambda tracks: tracks.pt > 1
    select_match = lambda tracks: tracks.matched >= 1
    select_eta = lambda tracks: (tracks.eta < -2.5) & (tracks.eta > -4)
    select_Rabs = lambda tracks: (D_ABS * np.abs(np.tan(tracks.thetaAbs * (3.141592/180))) > 17.5) & (D_ABS * np.abs(np.tan(tracks.thetaAbs*(3.141592/180))) < 90)
    
    # Fonction pour calculer la masse invariante SANS condition sur pDCA
    def calculate_invariant_mass(seltracks, use_pDCA=False):
        invariant_mass = []
        with tqdm(total=len(seltracks.to_list()), desc="Calculating...", bar_format='{l_bar}{bar:30}{r_bar}{bar:-30b}') as bar:
            for i, eventTracks in enumerate(seltracks):
                bar.update(1)
                if len(eventTracks) < 2: 
                    continue
                #print(i)
                #print(seltracks[18].to_list())
                #print(seltracks[20].to_list())
                
    
                
                positive = eventTracks[eventTracks.charge == 1]
                negative = eventTracks[eventTracks.charge == -1]
                
                matched_positive = set()
                matched_negative = set()
                
                for idx_p, p in enumerate(positive):
                    for idx_n, n in enumerate(negative):
                        if idx_p in matched_positive or idx_n in matched_negative: 
                            continue
                        
                        # Condition sur pDCA uniquement si demandé ET si le champ existe
                        if use_pDCA:
                            y = 0.5*np.log(x)((p.E+n.E + p.pz+n.pz)/(p.E+n.E - p.pz-n.pz))
                            if y>-4 and y<-2.5:
                                invariant_mass.append(np.sqrt(2*0.105*0.105 + 2*p.dot(n)))
                                matched_positive.add(idx_p)
                                matched_negative.add(idx_n)
                        else:
                            invariant_mass.append(np.sqrt(2*0.105*0.105 + 2*p.dot(n)))
                            matched_positive.add(idx_p)
                            matched_negative.add(idx_n)
        return invariant_mass
    
    # === SÉLECTIONS SUR LES EVENTS ===
    print("Sélection nMuons >= 2...")
    goodEvents = applySelection(events, select_nb, plot=False)
    
    print("Sélection zVtx...")
    goodEvents = applySelection(goodEvents, select_z, plot=False)
    
    print("Sélection charge...")
    goodEvents = applySelection(goodEvents, select_charge, plot=False)
    
    # === SÉLECTIONS SUR LES TRACKS ===
    tracks = getTracks(goodEvents)
    
    # 1. Coupure pT
    print("\n=== Coupure pT ===")
    tracks_pt = applySelection(tracks, select_pt, plot=False, title=r"$p_T$ selection")
    mass_pt = calculate_invariant_mass(tracks_pt, use_pDCA=False)
    hist_pt = hist.Hist(hist.axis.Regular(bins=1000, start=0, stop=5, name='$|p|$'))
    hist_pt.fill(mass_pt)
    Plot(hist_pt, title="Masse invariante - Coupure pT > 1 GeV", save=True)
    del mass_pt, hist_pt
    gc.collect()
    
    # 2. Coupure pT + match
    print("\n=== Coupure pT + match ===")
    tracks_match = applySelection(tracks_pt, select_match, plot=False, title="match selection")
    mass_match = calculate_invariant_mass(tracks_match, use_pDCA=False)
    hist_match = hist.Hist(hist.axis.Regular(bins=1000, start=0, stop=5, name='$|p|$'))
    hist_match.fill(mass_match)
    Plot(hist_match, title="Masse invariante - Coupures pT + match", save=True)
    del mass_match, hist_match
    gc.collect()
    
    # 3. Coupure pT + match + eta
    print("\n=== Coupure pT + match + eta ===")
    tracks_eta = applySelection(tracks_match, select_eta, plot=False, title=r"$\eta$ selection")
    mass_eta = calculate_invariant_mass(tracks_eta, use_pDCA=False)
    hist_eta = hist.Hist(hist.axis.Regular(bins=1000, start=0, stop=5, name='$|p|$'))
    hist_eta.fill(mass_eta)
    Plot(hist_eta, title=r"Masse invariante - Coupures pT + match + $\eta$", save=True)
    del mass_eta, hist_eta
    gc.collect()
    
    # 4. Coupure pT + match + eta + Rabs
    print("\n=== Coupure pT + match + eta + Rabs ===")
    tracks_Rabs = applySelection(tracks_eta, select_Rabs, plot=False, title=r"$R_abs$ selection")
    mass_Rabs = calculate_invariant_mass(tracks_Rabs, use_pDCA=False)
    hist_Rabs = hist.Hist(hist.axis.Regular(bins=1000, start=0, stop=5, name='$|p|$'))
    hist_Rabs.fill(mass_Rabs)
    Plot(hist_Rabs, title=r"Masse invariante - Coupures pT + match + $\eta$ + $R_{abs}$", save=True)
    del mass_Rabs, hist_Rabs
    gc.collect()
    
    # 5. Coupures (pT + match + eta + Rabs + pDCA)
    print("\n=== Coupure pT + match + eta + Rabs + pDCA (single muons) ===")
    pDCA, N, sigmapDCA = get_pDCA_infos(tracks_Rabs)
    select_pDCA = lambda tracks: pDCA < N*sigmapDCA
    goodTracks = applySelection(tracks_Rabs, select_pDCA, plot=False, title="pDCA selection")
    mass_pDCA_single = calculate_invariant_mass(goodTracks, use_pDCA=False)
    vhMagRaw.fill(mass_pDCA_single)
    Plot(vhMagRaw, title="Masse invariante - Coupures pT + match + $\eta$ + $R_{abs}$ + pDCA (single)", save=True)


    # 6. TOUTES les coupures (pT + match + eta + Rabs + pDCA)
    print("\n=== Coupure pT + match + eta + Rabs + pDCA (single muons) + pDCA (pairs)===")
    mass_final = calculate_invariant_mass(goodTracks, use_pDCA=True)
    vhMagRaw.fill(mass_final)
    Plot(vhMagRaw, title="Masse invariante - TOUTES les coupures", save=True)
    
    # Nettoyage final
    del events, goodEvents, tracks, tracks_pt, tracks_match, tracks_eta, tracks_Rabs, goodTracks, mass_final
    gc.collect()
    
    print("\n✓ Analyse terminée !")

<>:126: SyntaxWarning: invalid escape sequence '\e'
<>:126: SyntaxWarning: invalid escape sequence '\e'
/tmp/ipykernel_9664/2163857616.py:126: SyntaxWarning: invalid escape sequence '\e'
  Plot(vhMagRaw, title="Masse invariante - Coupures pT + match + $\eta$ + $R_{abs}$ + pDCA (single)", save=True)


In [6]:
analyzeEventsTree()

{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {291694} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290254} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290427} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290613} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290776} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {291002} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {291360} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {291482} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {291755} . {'data'} .root
{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {292061} . {'data'} .root
Sélection nMuons >= 2...
Sélection zVtx...
Sélection charge...

=== Coupure pT ===


Calculating...:  81%|████████████████████████▍     | 840372/1033216 [04:51<01:06, 2880.39it/s]
/tmp/ipykernel_9664/2163857616.py:126: SyntaxWarning: invalid escape sequence '\e'
  Plot(vhMagRaw, title="Masse invariante - Coupures pT + match + $\eta$ + $R_{abs}$ + pDCA (single)", save=True)


KeyboardInterrupt: 

# Fitting
## Using Roofit let's fit the background noise to extract $N_{J/\psi}$

In [ ]:
import ROOT as R

# Create observables
x = R.RooRealVar("x", "x", 1500, 5000)

left = 3050
right = 3150
# Construct signal pdf
mean = R.RooRealVar('mean', '', 3100, 3000, 3200)
sigma = R.RooRealVar('sigma', '', 100, 50, 150)
alpha = R.RooRealVar('alpha', '', 1, 0, 100)
n = R.RooRealVar('n', '', 3, 1, 100)

cbs = R.RooCBShape("CrystallBall", "CrystallBall", x, mean, sigma, alpha_1, n_1)

x.setRange("low", 1500, left)
x.setRange("high", right, 5000)
x.setRange("full", 1500, 5000)

fit_range = "low,high"



"""
sig = R.RooAddPdf('sig', '', R.RooArgList(cbs_1, cbs_2), R.RooArgList(mc_frac))
sig_yield = R.RooRealVar('sig_yield', '', 1794.4, 0, 100000)



exp = R.RooRealVar('exp', '', -2e-3, -4e-3, -0.01e-3)
combinatorial = R.RooExponential('combinatorial', '', x, exp)
bkg_yield = R.RooRealVar('bkg_yield', '', bkg, 0, werte)


pdf_cbWithExp = R.RooAddPdf(
    'pdf_cbWithExp',
    '',
    R.RooArgList(sig, combinatorial),
    R.RooArgList(sig_yield, bkg_yield)
)



data = R.RooDataSet(pdf_cbWithExp.generate(R.RooArgSet(x), werte))
data_red = data.reduce("x<{}||x>{}".format(left, right))

pdf_cbWithExp.fitTo(data_red, R.RooFit.Range(fit_range), R.RooFit.NumCPU(4), R.RooFit.Save(True), R.RooFit.Extended())
my_plot = x.frame()

data_red.plotOn(my_plot)
pdf_cbWithExp.plotOn(my_plot, R.RooFit.Range(fit_range), R.RooFit.NormRange(fit_range))
pdf_cbWithExp.getParameters(data).writeToFile("results.txt")

can = R.TCanvas()
can.SetLogy()
my_plot.Draw()

can.SaveAs("blind_test_ranges.pdf")
"""